# Connect to the cluster

In [ ]:
import socket
on_io = socket.gethostname() == 'io'
if on_io and 0:
    import hpc05, hpc05_monitor
    client = hpc05.connect.start_remote_and_connect(130, profile='pbs_15GB', folder='~/Work/two_dim_majoranas')[0]
#     max_usage_task = hpc05_monitor.start(client, interval=1)
else:
    # No not connect to the cluster in CI for example.
    client = None

## Numerics imports

In [ ]:
from functools import partial
from copy import copy
import numpy as np
import kwant
import adaptive
import zigzag
adaptive.notebook_extension()

import pathlib
pathlib.Path('paper/figures').mkdir(parents=True, exist_ok=True) 

## Plotting imports

In [ ]:
import matplotlib
matplotlib.use('agg')

%matplotlib inline
import matplotlib.pyplot as plt

## Parameter and system definitions

In [ ]:
import scipy.constants
import cmath

constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

default_params = dict(
    g_factor_middle=26, g_factor_left=0, g_factor_right=0,
    mu=10,
    alpha_middle=20, alpha_left=0, alpha_right=0,
    Delta_left=1, Delta_right=1,
    B_x=0,
    B_y=0,
    B_z=0,
    phase=np.pi,
    V=0,
    **constants)

default_syst_pars = dict(
    W=200,
    L_x=1300,
    L_sc_up=300,
    L_sc_down=300,
    z_x=1300,
    z_y=0,
    a=10,
    shape='parallel_curve',
    transverse_soi=True,
    mu_from_bottom_of_spin_orbit_bands=True,
    k_x_in_sc=True,
    wraparound=True,
    infinite=True,
    current=False,
    ns_junction=False)

# Figure 2. E_gap(z_y)

## Complete zigzag system

### Define learners

In [ ]:
import operator
from adaptive.learner.learner1D import curvature_loss_function

def gap_wrapper2(k_x, z_y, ratio, syst_pars=copy(default_syst_pars), params=copy(default_params), nbands=10):
    import numpy as np
    import zigzag
    z_x = z_y * ratio
    params = dict(params, k_x=k_x, mu=40, B_x=1)
    syst_pars = dict(syst_pars, z_y=z_y, z_x=z_x, L_x=z_x,
                     a=5, shape='sawtooth', L_sc_up=800, L_sc_down=800)
    syst = zigzag.system(**syst_pars)
    Es = zigzag.spectrum(syst, params, k=nbands)[0]
    return dict(E_min=np.abs(Es).min(), energies=Es)

def fnames(learner):
    val = learner.function.keywords  # because functools.partial
    fname = '__'.join([f'{k}_{v}.pickle' for k, v in val.items()])
    return 'data/z_y_vs_E_gap_mu40meV/' + fname

Learner = adaptive.make_datasaver(adaptive.Learner1D, arg_picker=operator.itemgetter('E_min'))
kwargs = dict(bounds=[-np.pi, np.pi], loss_per_interval=curvature_loss_function())
z_ys = np.arange(10, 405, 5)

combos = {'z_y': z_ys, 'ratio': [4, 8, 16]}
learner = adaptive.BalancingLearner.from_product(gap_wrapper2, Learner, kwargs, combos)
learner.strategy = 'npoints'
learner.load(fnames)

In [ ]:
runner = adaptive.Runner(learner, lambda l: l.learners[0].npoints > 95, executor=client,
                         retries=20, raise_if_retries_exceeded=False)
runner.live_info()

In [ ]:
saving_task = runner.start_periodic_saving(dict(fname=fnames), 600)

In [ ]:
# Plot the entire band structure
def plot_bands(l):
    import holoviews as hv
    if l.data:
        ks, vals = zip(*l.extra_data.items())
        Es = np.array([x['energies'] for x in vals])
        E_min = min(l.data.values())
        scatter = hv.Overlay([hv.Scatter((ks, E)).opts(style=dict(color='k')) for E in Es.T])
        return scatter * hv.HLine(E_min)
    else:
        return hv.Overlay(hv.Scatter([]))

learner.plot(plotter=plot_bands)

In [ ]:
from collections import defaultdict
d = defaultdict(dict)
for cdim, l in zip(learner._cdims_default, learner.learners):
    d[cdim['ratio']][cdim['z_y']] = l

def get_mins(learners_dict):
    learners = learners_dict.values()
    return [min(l.data.values()) for l in learners]

fig, ax = plt.subplots()
for ratio, val in d.items():
    E_gaps = get_mins(val)
    ax.plot(z_ys, E_gaps, label=f'ratio {ratio}')
ax.legend()
ax.set_xlabel('$z_y$ (nm)')
ax.set_ylabel(r'$E_{\textrm{gap}}$ (meV)')
plt.savefig('paper/figures/z_y_vs_E_gap.pdf')
plt.show()

In [ ]:
import pickle
with open('data/qc/tmp.pickle', 'wb') as f:
    E_gaps = {ratio: get_mins(val) for ratio, val in d.items()}
    pickle.dump([z_ys, E_gaps], f)

## Quasi classical system

In [ ]:
# "quasi classics"
a = 4
syst_pars = dict(
    default_syst_pars,
    L_sc_up=1600,
    L_sc_down=1600,
    L_x=a,
    z_x=a,
    a=a,
    shape=None,
    mu_from_bottom_of_spin_orbit_bands=False
)

params = dict(default_params, B=0.4, mu=40)

def bands(k_x, ratio, params=params, syst_pars=syst_pars):
    syst = zigzag.system(**syst_pars)
    theta = np.arctan(4 / ratio)
    params = dict(params,
                  theta=theta,
                  B_x=np.cos(theta)*params['B'],
                  B_y=np.sin(theta)*params['B'],
                  k_x=k_x*a)
    return np.min(np.abs(zigzag.spectrum(syst, params, k=4)[0]))

ratios = [4, 8, 16]
k_F = np.sqrt(params['mu'] * (2 * params['m_eff'])) / params['hbar']
loss = adaptive.learner.learner1D.triangle_loss

learner = adaptive.BalancingLearner.from_product(
    bands,
    learner_type=adaptive.Learner1D,
    learner_kwargs=dict(bounds=[0, 1.1*k_F], loss_per_interval=loss),
    combos=dict(ratio=ratios))
learner.strategy = 'npoints'

def fnames(learner):
    val = learner.function.keywords  # because functools.partial
    fname = '__'.join([f'{k}_{v}.pickle' for k, v in val.items()])
    return 'data/qc/' + fname

learner.load(fnames)

In [ ]:
runner = adaptive.Runner(learner, goal=lambda l: l.learners[0].npoints > 400)
runner.live_info()

In [ ]:
learner.plot()

In [ ]:
from types import SimpleNamespace
from scipy.optimize import minimize_scalar
from scipy.misc import derivative

def energies(k_y, params):
    p = SimpleNamespace(**params)
    g = p.g_factor_middle
    alpha = p.alpha_middle
    E_z = 0.5 * g * p.mu_B * p.B
    sqrt_term = np.sqrt(
        + E_z**2
        + alpha * (2 * E_z * (k_y * np.cos(p.theta) - p.k_x * np.sin(p.theta))
                   + alpha * (p.k_x**2 + k_y**2))
    )
    term = p.hbar**2 * (p.k_x**2 + k_y**2) / (2 * p.m_eff) - p.mu
    return [-sqrt_term + term, sqrt_term + term]

def find_k_ys(params):
    f = lambda k_y, i: abs(energies(k_y, params)[i])
    return [minimize_scalar(f, args=i, bounds=[0, np.pi], method='bounded')['x']
            for i in [0, 1]]

def find_dE_dkys(k_ys, params):
    f = lambda k_y, params, i: energies(k_y, params)[i]
    return [derivative(f, k_y, dx=1e-6, args=[params, i])
            for i, k_y in enumerate(k_ys)]

def find_dE_dkxs(k_ys, params):
    def _f(k_x, k_y, params, i):
        params['k_x'] = float(k_x)
        return energies(k_y, params)[i]
    return [derivative(_f, params['k_x'], dx=1e-6, args=[k_y, copy(params), i])
            for i, k_y in enumerate(k_ys)]

def tan_a(params):
    k_ys = find_k_ys(params)
    dE_dkx = find_dE_dkxs(k_ys, params)
    dE_dky = find_dE_dkys(k_ys, params)
    return np.array(dE_dky) / np.array(dE_dkx)

def a_min(z_x, z_y, W, *, correction=True):
    theta = np.arctan(4 * z_y / z_x)
    crossing_point = W / np.cos(theta)
    if z_y < crossing_point / 2:
        return np.nan
    a = z_x / 2
    b = z_y * 2
    c = np.sqrt(W**2 + (b/a)**2 * W**2)
    
    if correction:
        D = np.sqrt(a**2 + (b - c)**2)
    else:
        D = np.sqrt(a**2 + (b + c)**2)
    xprime = np.sqrt(D**2 - W**2)
    return W / xprime

def get_cutoff(z_x, z_y, W, params):
    a_m = a_min(z_x, z_y, W)
    
    k_F = np.sqrt(params['mu'] * (2 * params['m_eff'])) / params['hbar']
    
    def f(k_x):
        params['k_x'] = k_x
        return abs(np.min(np.abs(tan_a(params))) - a_m)

    k_c, e = scipy.optimize.fmin(f, k_F/2, ftol=1e-7, xtol=1e-6, full_output=1, disp=0)[:2]
    return k_c if e < 1e-3 else np.inf

In [ ]:
def get_gaps(z_ys, learner, ratio, syst_pars=syst_pars):
    ks, Es = zip(*sorted(learner.data.items()))
    W = syst_pars['W']
    params['theta'] = np.arctan(4 / ratio)
    k_max_at_z_y = np.array([get_cutoff(ratio*z_y, z_y, W, params) for z_y in z_ys])
    E_gaps = np.interp(k_max_at_z_y, ks, np.minimum.accumulate(Es, 0))
    return E_gaps

z_ys = np.linspace(100, 400, 100)
data = {cdim['ratio']: get_gaps(z_ys, l, cdim['ratio'])
        for cdim, l in zip(learner._cdims_default, learner.learners)}

In [ ]:
fig, ax = plt.subplots()
for ratio, E_gaps in data.items():
    ax.plot(z_ys, E_gaps, label=f'ratio {ratio}, QC')

ax.legend()
ax.set_xlabel('$z_y$ (nm)')
ax.set_ylabel(r'$E_{\textrm{gap}}$ (meV)')
plt.savefig('paper/figures/z_y_vs_E_gap2.pdf')
plt.show()

In [ ]:
import pickle
with open('data/qc/tmp_quasi.pickle', 'wb') as f:
    pickle.dump([z_ys, data], f)

In [ ]:
import pickle
with open('data/qc/tmp.pickle', 'rb') as f:
    z_ys_complete, E_gaps_complete = pickle.load(f)
    
with open('data/qc/tmp_quasi.pickle', 'rb') as f:
    z_ys_quasi, E_gaps_quasi = pickle.load(f)

In [ ]:
fig, ax = plt.subplots()

col = {4: 'b', 8: 'g', 16: 'red'}

for ratio, E_gaps in E_gaps_quasi.items():
    ax.plot(z_ys_quasi, E_gaps, ls='--', c=col[ratio])

for ratio, E_gaps in E_gaps_complete.items():
    ax.plot(z_ys_complete, E_gaps, c=col[ratio], label=f'ratio {ratio}')
    
ax.legend()
ax.set_xlabel('$z_y$ (nm)')
ax.set_ylabel(r'$E_{\textrm{gap}}$ (meV)')
plt.savefig('paper/figures/z_y_vs_E_gap2.pdf')
plt.show()

In [ ]:
# Fermi wavelength
mu = 20
mu = mu * scipy.constants.eV * 1e-3
m = 0.02 * scipy.constants.m_e
hbar = scipy.constants.hbar
k_F = np.sqrt(2 * mu * m / hbar ** 2)
(2 * np.pi / k_F) * 1e9